In [1]:
import torch

print("number of gpu:", torch.cuda.device_count())
print("gpu name:", torch.cuda.get_device_name())

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

number of gpu: 1
gpu name: NVIDIA RTX A6000
Using device: cuda


# 1 VGG

## 1.1 VGG16

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import copy
import os

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
FROZEN_EPOCHS = 30
FINE_TUNE_EPOCHS = 70
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
FINE_TUNE_LR = 1e-5
BEST_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/new/Vgg16_ft_best_model.pth'
FINAL_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/new/Vgg16_ft_final_model.pth'

# Transforms
transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset & Dataloaders (Updated paths)
train_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAfricantoAmerican/train', transform=transform)
val_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/valid', transform=transform)
test_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Load pretrained VGG19 and freeze feature layers
base_model = models.vgg16(pretrained=True)
for param in base_model.features.parameters():
    param.requires_grad = False

# Replace only the classifier (keep avgpool as is)
base_model.classifier = nn.Sequential(
    nn.Linear(512 * 7 * 7, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 1),
    nn.Sigmoid()
)

model = base_model.to(device)

# Loss & optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=5, path=BEST_MODEL_PATH):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.best_model = None
        self.path = path

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
            torch.save(self.best_model, self.path)
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# Evaluation Function
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
    return total_loss / len(dataloader.dataset)

# Accuracy Function
def compute_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            preds = (outputs > 0.5).int().squeeze()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
    early_stopping = EarlyStopping(patience=5)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        val_loss = evaluate(model, val_loader, criterion)
        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}")

        if early_stopping(val_loss, model):
            print("Early stopping triggered.")
            break

# Train model with frozen layers
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FROZEN_EPOCHS)

# Load best model before fine-tuning
model.load_state_dict(torch.load(BEST_MODEL_PATH))

# Unfreeze last 50 conv layers
for param in list(model.features.parameters())[-50:]:
    param.requires_grad = True

# Recompile optimizer with updated params
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=FINE_TUNE_LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Fine-tune the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FINE_TUNE_EPOCHS)

# Save final model
torch.save(model.state_dict(), FINAL_MODEL_PATH)
print(f"✅ Final fine-tuned model saved at: {FINAL_MODEL_PATH}")

# Evaluate on test set
test_acc = compute_accuracy(model, test_loader)
print(f"✅ Test Accuracy after Fine-Tuning: {test_acc:.4f}")


c:\Users\vb11574\AppData\Local\miniconda3\envs\salmo\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\vb11574\AppData\Local\miniconda3\envs\salmo\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/30, Validation Loss: 0.6385
Epoch 2/30, Validation Loss: 0.6416
Epoch 3/30, Validation Loss: 0.6236
Epoch 4/30, Validation Loss: 0.6578
Epoch 5/30, Validation Loss: 0.6622
Epoch 6/30, Validation Loss: 0.6913
Epoch 7/30, Validation Loss: 0.7156
Epoch 8/30, Validation Loss: 0.7141
Early stopping triggered.
Epoch 1/70, Validation Loss: 0.6746
Epoch 2/70, Validation Loss: 0.7653
Epoch 3/70, Validation Loss: 0.7863
Epoch 4/70, Validation Loss: 0.9367
Epoch 5/70, Validation Loss: 0.9944
Epoch 6/70, Validation Loss: 1.0492
Early stopping triggered.
✅ Final fine-tuned model saved at: C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/Vgg16_ft_final_model.pth
✅ Test Accuracy after Fine-Tuning: 0.3886


## 1.2 VGG19

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import copy
import os

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
FROZEN_EPOCHS = 30
FINE_TUNE_EPOCHS = 70
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
FINE_TUNE_LR = 1e-5
BEST_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/Vgg19_ft_best_model.pth'
FINAL_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/Vgg19_ft_final_model.pth'

# Transforms
transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset & Dataloaders
train_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/train', transform=transform)
val_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/valid', transform=transform)
test_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Load pretrained VGG19 and freeze feature layers
base_model = models.vgg19(pretrained=True)
for param in base_model.features.parameters():
    param.requires_grad = False

# Replace only the classifier (keep avgpool as is)
base_model.classifier = nn.Sequential(
    nn.Linear(512 * 7 * 7, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 1),
    nn.Sigmoid()
)

model = base_model.to(device)

# Loss & optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=5, path=BEST_MODEL_PATH):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.best_model = None
        self.path = path

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
            torch.save(self.best_model, self.path)
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# Evaluation Function
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
    return total_loss / len(dataloader.dataset)

# Accuracy Function
def compute_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            preds = (outputs > 0.5).int().squeeze()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
    early_stopping = EarlyStopping(patience=5)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        val_loss = evaluate(model, val_loader, criterion)
        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}")

        if early_stopping(val_loss, model):
            print("Early stopping triggered.")
            break

# Train model with frozen layers
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FROZEN_EPOCHS)

# Load best model before fine-tuning
model.load_state_dict(torch.load(BEST_MODEL_PATH))

# Unfreeze last 50 conv layers
for param in list(model.features.parameters())[-50:]:
    param.requires_grad = True

# Recompile optimizer with updated params
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=FINE_TUNE_LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Fine-tune the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FINE_TUNE_EPOCHS)

# Save final model
torch.save(model.state_dict(), FINAL_MODEL_PATH)
print(f"✅ Final fine-tuned model saved at: {FINAL_MODEL_PATH}")

# Evaluate on test set
test_acc = compute_accuracy(model, test_loader)
print(f"✅ Test Accuracy after Fine-Tuning: {test_acc:.4f}")

c:\Users\vb11574\AppData\Local\miniconda3\envs\salmo\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/30, Validation Loss: 0.6497
Epoch 2/30, Validation Loss: 0.6509
Epoch 3/30, Validation Loss: 0.6538
Epoch 4/30, Validation Loss: 0.6966
Epoch 5/30, Validation Loss: 0.6902
Epoch 6/30, Validation Loss: 0.6701
Early stopping triggered.
Epoch 1/70, Validation Loss: 0.6338
Epoch 2/70, Validation Loss: 0.6609
Epoch 3/70, Validation Loss: 0.7250
Epoch 4/70, Validation Loss: 0.8342
Epoch 5/70, Validation Loss: 1.1709
Epoch 6/70, Validation Loss: 1.0216
Early stopping triggered.
✅ Final fine-tuned model saved at: C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/Vgg19_ft_final_model.pth
✅ Test Accuracy after Fine-Tuning: 0.3185


# 2 MobileNet

## 2.1 MobileNet v2

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import copy
import os

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
FROZEN_EPOCHS = 30
FINE_TUNE_EPOCHS = 70
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
FINE_TUNE_LR = 1e-5
BEST_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/MobileNetV2_ft_best_model.pth'
FINAL_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/MobileNetV2_ft_final_model.pth'

# Transforms
transform = Compose([
    Resize((224, 224)),  # MobileNetV1 typically uses 224x224 images
    ToTensor(),
    Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset & Dataloaders (Updated paths)
train_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/train', transform=transform)
val_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/valid', transform=transform)
test_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Load pretrained MobileNetV1 and freeze feature layers
base_model = models.mobilenet_v2(pretrained=True)  # MobileNetV1 equivalent is available as MobileNetV2 in torchvision
for param in base_model.parameters():
    param.requires_grad = False  # Freeze all layers initially

# Replace the classifier part of the model (keep global average pooling)
base_model.classifier = nn.Sequential(
    nn.Linear(base_model.classifier[1].in_features, 512),  # base_model.classifier[1].in_features gives the correct input size
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 1),
    nn.Sigmoid()
)

model = base_model.to(device)

# Loss & optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=5, path=BEST_MODEL_PATH):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.best_model = None
        self.path = path

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
            torch.save(self.best_model, self.path)
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# Evaluation Function
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
    return total_loss / len(dataloader.dataset)

# Accuracy Function
def compute_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            preds = (outputs > 0.5).int().squeeze()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
    early_stopping = EarlyStopping(patience=5)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        val_loss = evaluate(model, val_loader, criterion)
        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}")

        if early_stopping(val_loss, model):
            print("Early stopping triggered.")
            break

# Train model with frozen layers
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FROZEN_EPOCHS)

# Load best model before fine-tuning
model.load_state_dict(torch.load(BEST_MODEL_PATH))

# Unfreeze all layers in the classifier and some feature layers
for param in list(model.features.parameters())[-50:]:
    param.requires_grad = True

# Recompile optimizer with updated params
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=FINE_TUNE_LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Fine-tune the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FINE_TUNE_EPOCHS)

# Save final model
torch.save(model.state_dict(), FINAL_MODEL_PATH)
print(f"✅ Final fine-tuned model saved at: {FINAL_MODEL_PATH}")

# Evaluate on test set
test_acc = compute_accuracy(model, test_loader)
print(f"✅ Test Accuracy after Fine-Tuning: {test_acc:.4f}")


c:\Users\vb11574\AppData\Local\miniconda3\envs\salmo\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\vb11574/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100.0%


Epoch 1/30, Validation Loss: 0.6553
Epoch 2/30, Validation Loss: 0.6463
Epoch 3/30, Validation Loss: 0.6428
Epoch 4/30, Validation Loss: 0.6384
Epoch 5/30, Validation Loss: 0.6390
Epoch 6/30, Validation Loss: 0.6411
Epoch 7/30, Validation Loss: 0.6314
Epoch 8/30, Validation Loss: 0.6292
Epoch 9/30, Validation Loss: 0.6554
Epoch 10/30, Validation Loss: 0.6309
Epoch 11/30, Validation Loss: 0.6245
Epoch 12/30, Validation Loss: 0.6252
Epoch 13/30, Validation Loss: 0.6330
Epoch 14/30, Validation Loss: 0.6280
Epoch 15/30, Validation Loss: 0.6247
Epoch 16/30, Validation Loss: 0.6250
Early stopping triggered.
Epoch 1/70, Validation Loss: 0.6216
Epoch 2/70, Validation Loss: 0.6149
Epoch 3/70, Validation Loss: 0.6129
Epoch 4/70, Validation Loss: 0.6094
Epoch 5/70, Validation Loss: 0.6093
Epoch 6/70, Validation Loss: 0.6076
Epoch 7/70, Validation Loss: 0.6068
Epoch 8/70, Validation Loss: 0.6105
Epoch 9/70, Validation Loss: 0.6160
Epoch 10/70, Validation Loss: 0.6178
Epoch 11/70, Validation Loss: 

## 2.2 MobileNet v3

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import copy
import os

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
FROZEN_EPOCHS = 30
FINE_TUNE_EPOCHS = 70
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
FINE_TUNE_LR = 1e-5
BEST_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/MobileNetV3_ft_best_model.pth'
FINAL_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/MobileNetV3_ft_final_model.pth'

# Transforms
transform = Compose([
    Resize((224, 224)),  # MobileNetV3 typically uses 224x224 images
    ToTensor(),
    Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset & Dataloaders (Updated paths)
train_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/train', transform=transform)
val_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/valid', transform=transform)
test_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Load pretrained MobileNetV3 and freeze feature layers
base_model = models.mobilenet_v3_large(pretrained=True)  # MobileNetV3
base_model = base_model.to(device)  # Move the model to the GPU (if available)

for param in base_model.parameters():
    param.requires_grad = False  # Freeze all layers initially

# Dummy input to get the number of input features to the classifier
dummy_input = torch.randn(1, 3, 224, 224).to(device)  # Example input with the same size as the input images
feature_output = base_model.features(dummy_input)  # Get output from the feature extractor
feature_output = base_model.avgpool(feature_output)  # Apply average pooling
feature_output = torch.flatten(feature_output, 1)  # Flatten the pooled output
num_ftrs = feature_output.size(1)  # The number of features (flattened size)

# Replace the classifier part of the model (keep global average pooling)
base_model.classifier = nn.Sequential(
    nn.Linear(num_ftrs, 512),  # Use num_ftrs for the input size of the linear layer
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 1),
    nn.Sigmoid()
)

model = base_model.to(device)  # Ensure the model is on the same device

# Loss & optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=5, path=BEST_MODEL_PATH):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.best_model = None
        self.path = path

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
            torch.save(self.best_model, self.path)
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# Evaluation Function
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
    return total_loss / len(dataloader.dataset)

# Accuracy Function
def compute_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            preds = (outputs > 0.5).int().squeeze()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
    early_stopping = EarlyStopping(patience=5)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        val_loss = evaluate(model, val_loader, criterion)
        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}")

        if early_stopping(val_loss, model):
            print("Early stopping triggered.")
            break

# Train model with frozen layers
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FROZEN_EPOCHS)

# Load best model before fine-tuning
model.load_state_dict(torch.load(BEST_MODEL_PATH))

# Unfreeze all layers in the classifier and some feature layers
for param in list(model.features.parameters())[-50:]:
    param.requires_grad = True

# Recompile optimizer with updated params
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=FINE_TUNE_LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Fine-tune the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FINE_TUNE_EPOCHS)

# Save final model
torch.save(model.state_dict(), FINAL_MODEL_PATH)
print(f"✅ Final fine-tuned model saved at: {FINAL_MODEL_PATH}")

# Evaluate on test set
test_acc = compute_accuracy(model, test_loader)
print(f"✅ Test Accuracy after Fine-Tuning: {test_acc:.4f}")


Epoch 1/30, Validation Loss: 0.6655
Epoch 2/30, Validation Loss: 0.6499
Epoch 3/30, Validation Loss: 0.6420
Epoch 4/30, Validation Loss: 0.6444
Epoch 5/30, Validation Loss: 0.6386
Epoch 6/30, Validation Loss: 0.6342
Epoch 7/30, Validation Loss: 0.6510
Epoch 8/30, Validation Loss: 0.6304
Epoch 9/30, Validation Loss: 0.6466
Epoch 10/30, Validation Loss: 0.6252
Epoch 11/30, Validation Loss: 0.6286
Epoch 12/30, Validation Loss: 0.6257
Epoch 13/30, Validation Loss: 0.6285
Epoch 14/30, Validation Loss: 0.6235
Epoch 15/30, Validation Loss: 0.6297
Epoch 16/30, Validation Loss: 0.6482
Epoch 17/30, Validation Loss: 0.6249
Epoch 18/30, Validation Loss: 0.6145
Epoch 19/30, Validation Loss: 0.6149
Epoch 20/30, Validation Loss: 0.6136
Epoch 21/30, Validation Loss: 0.6186
Epoch 22/30, Validation Loss: 0.6115
Epoch 23/30, Validation Loss: 0.6113
Epoch 24/30, Validation Loss: 0.6105
Epoch 25/30, Validation Loss: 0.6082
Epoch 26/30, Validation Loss: 0.6037
Epoch 27/30, Validation Loss: 0.6259
Epoch 28/3

# 3 DenseNet

## 3.1 DenseNet 121

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import copy
import os

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
FROZEN_EPOCHS = 30
FINE_TUNE_EPOCHS = 70
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
FINE_TUNE_LR = 1e-5
BEST_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/DenseNet121_ft_best_model.pth'
FINAL_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/DenseNet121_ft_final_model.pth'

# Transforms
transform = Compose([
    Resize((224, 224)),  # DenseNet121 typically uses 224x224 images
    ToTensor(),
    Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset & Dataloaders (Updated paths)
train_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/train', transform=transform)
val_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/valid', transform=transform)
test_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Load pretrained DenseNet121 and freeze feature layers
base_model = models.densenet121(pretrained=True)  # DenseNet121
base_model = base_model.to(device)  # Ensure the model is on the correct device

for param in base_model.parameters():
    param.requires_grad = False  # Freeze all layers initially

# Get the correct number of features from the classifier layer
num_ftrs = base_model.classifier.in_features  # Access the input features to the first Linear layer

# Replace the classifier part of the model (keep global average pooling)
base_model.classifier = nn.Sequential(
    nn.Linear(num_ftrs, 512),  # Use num_ftrs for the input size of the linear layer
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 1),
    nn.Sigmoid()
)

model = base_model.to(device)  # Ensure the model is on the correct device

# Loss & optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=5, path=BEST_MODEL_PATH):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.best_model = None
        self.path = path

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
            torch.save(self.best_model, self.path)
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# Evaluation Function
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
    return total_loss / len(dataloader.dataset)

# Accuracy Function
def compute_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            preds = (outputs > 0.5).int().squeeze()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
    early_stopping = EarlyStopping(patience=5)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        val_loss = evaluate(model, val_loader, criterion)
        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}")

        if early_stopping(val_loss, model):
            print("Early stopping triggered.")
            break

# Train model with frozen layers
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FROZEN_EPOCHS)

# Load best model before fine-tuning
model.load_state_dict(torch.load(BEST_MODEL_PATH))

# Unfreeze all layers in the classifier and some feature layers
for param in list(model.features.parameters())[-50:]:
    param.requires_grad = True

# Recompile optimizer with updated params
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=FINE_TUNE_LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Fine-tune the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FINE_TUNE_EPOCHS)

# Save final model
torch.save(model.state_dict(), FINAL_MODEL_PATH)
print(f"✅ Final fine-tuned model saved at: {FINAL_MODEL_PATH}")

# Evaluate on test set
test_acc = compute_accuracy(model, test_loader)
print(f"✅ Test Accuracy after Fine-Tuning: {test_acc:.4f}")


Epoch 1/30, Validation Loss: 0.6567
Epoch 2/30, Validation Loss: 0.6481
Epoch 3/30, Validation Loss: 0.6427
Epoch 4/30, Validation Loss: 0.6387
Epoch 5/30, Validation Loss: 0.6368
Epoch 6/30, Validation Loss: 0.6306
Epoch 7/30, Validation Loss: 0.6298
Epoch 8/30, Validation Loss: 0.6470
Epoch 9/30, Validation Loss: 0.6324
Epoch 10/30, Validation Loss: 0.6246
Epoch 11/30, Validation Loss: 0.6305
Epoch 12/30, Validation Loss: 0.6250
Epoch 13/30, Validation Loss: 0.6273
Epoch 14/30, Validation Loss: 0.6284
Epoch 15/30, Validation Loss: 0.6188
Epoch 16/30, Validation Loss: 0.6189
Epoch 17/30, Validation Loss: 0.6188
Epoch 18/30, Validation Loss: 0.6169
Epoch 19/30, Validation Loss: 0.6159
Epoch 20/30, Validation Loss: 0.6203
Epoch 21/30, Validation Loss: 0.6179
Epoch 22/30, Validation Loss: 0.6201
Epoch 23/30, Validation Loss: 0.6172
Epoch 24/30, Validation Loss: 0.6178
Early stopping triggered.
Epoch 1/70, Validation Loss: 0.6155
Epoch 2/70, Validation Loss: 0.6159
Epoch 3/70, Validation 

## 3.2 DenseNet 169

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import copy
import os

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
FROZEN_EPOCHS = 30
FINE_TUNE_EPOCHS = 70
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
FINE_TUNE_LR = 1e-5
BEST_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/DenseNet121_ft_best_model.pth'
FINAL_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/DenseNet121_ft_final_model.pth'

# Transforms
transform = Compose([
    Resize((224, 224)),  # DenseNet121 typically uses 224x224 images
    ToTensor(),
    Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset & Dataloaders (Updated paths)
train_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/train', transform=transform)
val_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/valid', transform=transform)
test_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Load pretrained DenseNet121 and freeze feature layers
base_model = models.densenet169(pretrained=True)  # DenseNet121
base_model = base_model.to(device)  # Ensure the model is on the correct device

for param in base_model.parameters():
    param.requires_grad = False  # Freeze all layers initially

# Get the correct number of features from the classifier layer
num_ftrs = base_model.classifier.in_features  # Access the input features to the first Linear layer

# Replace the classifier part of the model (keep global average pooling)
base_model.classifier = nn.Sequential(
    nn.Linear(num_ftrs, 512),  # Use num_ftrs for the input size of the linear layer
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 1),
    nn.Sigmoid()
)

model = base_model.to(device)  # Ensure the model is on the correct device

# Loss & optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=5, path=BEST_MODEL_PATH):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.best_model = None
        self.path = path

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
            torch.save(self.best_model, self.path)
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# Evaluation Function
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
    return total_loss / len(dataloader.dataset)

# Accuracy Function
def compute_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            preds = (outputs > 0.5).int().squeeze()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
    early_stopping = EarlyStopping(patience=5)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        val_loss = evaluate(model, val_loader, criterion)
        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}")

        if early_stopping(val_loss, model):
            print("Early stopping triggered.")
            break

# Train model with frozen layers
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FROZEN_EPOCHS)

# Load best model before fine-tuning
model.load_state_dict(torch.load(BEST_MODEL_PATH))

# Unfreeze all layers in the classifier and some feature layers
for param in list(model.features.parameters())[-50:]:
    param.requires_grad = True

# Recompile optimizer with updated params
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=FINE_TUNE_LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Fine-tune the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FINE_TUNE_EPOCHS)

# Save final model
torch.save(model.state_dict(), FINAL_MODEL_PATH)
print(f"✅ Final fine-tuned model saved at: {FINAL_MODEL_PATH}")

# Evaluate on test set
test_acc = compute_accuracy(model, test_loader)
print(f"✅ Test Accuracy after Fine-Tuning: {test_acc:.4f}")


c:\Users\vb11574\AppData\Local\miniconda3\envs\salmo\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\vb11574\AppData\Local\miniconda3\envs\salmo\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet169_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet169_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to C:\Users\vb11574/.cache\torch\hub\checkpoints\densenet169-b2777c0a.pth
100.0%


Epoch 1/30, Validation Loss: 0.6566
Epoch 2/30, Validation Loss: 0.6542
Epoch 3/30, Validation Loss: 0.6475
Epoch 4/30, Validation Loss: 0.6822
Epoch 5/30, Validation Loss: 0.6263
Epoch 6/30, Validation Loss: 0.6223
Epoch 7/30, Validation Loss: 0.6179
Epoch 8/30, Validation Loss: 0.6226
Epoch 9/30, Validation Loss: 0.6155
Epoch 10/30, Validation Loss: 0.6109
Epoch 11/30, Validation Loss: 0.6439
Epoch 12/30, Validation Loss: 0.6076
Epoch 13/30, Validation Loss: 0.6067
Epoch 14/30, Validation Loss: 0.6093
Epoch 15/30, Validation Loss: 0.6161
Epoch 16/30, Validation Loss: 0.6058
Epoch 17/30, Validation Loss: 0.6142
Epoch 18/30, Validation Loss: 0.6139
Epoch 19/30, Validation Loss: 0.5999
Epoch 20/30, Validation Loss: 0.5994
Epoch 21/30, Validation Loss: 0.5969
Epoch 22/30, Validation Loss: 0.5994
Epoch 23/30, Validation Loss: 0.6219
Epoch 24/30, Validation Loss: 0.5974
Epoch 25/30, Validation Loss: 0.6004
Epoch 26/30, Validation Loss: 0.6021
Early stopping triggered.
Epoch 1/70, Validatio

## 3.3 DenseNet 201

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import copy
import os

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
FROZEN_EPOCHS = 30
FINE_TUNE_EPOCHS = 70
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
FINE_TUNE_LR = 1e-5
BEST_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/DenseNet201_ft_best_model.pth'
FINAL_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/DenseNet201_ft_final_model.pth'

# Transforms
transform = Compose([
    Resize((224, 224)),  # MobileNetV1 typically uses 224x224 images
    ToTensor(),
    Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset & Dataloaders (Updated paths)
train_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/train', transform=transform)
val_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/valid', transform=transform)
test_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Load pretrained MobileNetV1 and freeze feature layers
base_model = models.densenet201(pretrained=True)  # MobileNetV1 equivalent is available as MobileNetV2 in torchvision
base_model = base_model.to(device)  # Ensure the model is on the correct device

for param in base_model.parameters():
    param.requires_grad = False  # Freeze all layers initially

# Get the correct number of features from the classifier layer
num_ftrs = base_model.classifier.in_features  # Access the input features to the first Linear layer

# Replace the classifier part of the model (keep global average pooling)
base_model.classifier = nn.Sequential(
    nn.Linear(num_ftrs, 512),  # base_model.classifier[1].in_features gives the correct input size
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 1),
    nn.Sigmoid()
)

model = base_model.to(device)

# Loss & optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=5, path=BEST_MODEL_PATH):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.best_model = None
        self.path = path

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
            torch.save(self.best_model, self.path)
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# Evaluation Function
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
    return total_loss / len(dataloader.dataset)

# Accuracy Function
def compute_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            preds = (outputs > 0.5).int().squeeze()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
    early_stopping = EarlyStopping(patience=5)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        val_loss = evaluate(model, val_loader, criterion)
        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}")

        if early_stopping(val_loss, model):
            print("Early stopping triggered.")
            break

# Train model with frozen layers
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FROZEN_EPOCHS)

# Load best model before fine-tuning
model.load_state_dict(torch.load(BEST_MODEL_PATH))

# Unfreeze all layers in the classifier and some feature layers
for param in list(model.features.parameters())[-50:]:
    param.requires_grad = True

# Recompile optimizer with updated params
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=FINE_TUNE_LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Fine-tune the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FINE_TUNE_EPOCHS)

# Save final model
torch.save(model.state_dict(), FINAL_MODEL_PATH)
print(f"✅ Final fine-tuned model saved at: {FINAL_MODEL_PATH}")

# Evaluate on test set
test_acc = compute_accuracy(model, test_loader)
print(f"✅ Test Accuracy after Fine-Tuning: {test_acc:.4f}")


c:\Users\vb11574\AppData\Local\miniconda3\envs\salmo\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to C:\Users\vb11574/.cache\torch\hub\checkpoints\densenet201-c1103571.pth
100.0%


Epoch 1/30, Validation Loss: 0.6587
Epoch 2/30, Validation Loss: 0.6500
Epoch 3/30, Validation Loss: 0.6431
Epoch 4/30, Validation Loss: 0.6365
Epoch 5/30, Validation Loss: 0.6342
Epoch 6/30, Validation Loss: 0.6403
Epoch 7/30, Validation Loss: 0.6287
Epoch 8/30, Validation Loss: 0.6260
Epoch 9/30, Validation Loss: 0.6242
Epoch 10/30, Validation Loss: 0.6418
Epoch 11/30, Validation Loss: 0.6280
Epoch 12/30, Validation Loss: 0.6246
Epoch 13/30, Validation Loss: 0.6267
Epoch 14/30, Validation Loss: 0.6241
Epoch 15/30, Validation Loss: 0.6244
Epoch 16/30, Validation Loss: 0.6255
Epoch 17/30, Validation Loss: 0.6235
Epoch 18/30, Validation Loss: 0.6222
Epoch 19/30, Validation Loss: 0.6226
Epoch 20/30, Validation Loss: 0.6230
Epoch 21/30, Validation Loss: 0.6245
Epoch 22/30, Validation Loss: 0.6250
Epoch 23/30, Validation Loss: 0.6247
Early stopping triggered.
Epoch 1/70, Validation Loss: 0.6217
Epoch 2/70, Validation Loss: 0.6187
Epoch 3/70, Validation Loss: 0.6192
Epoch 4/70, Validation L

# 4 Inception V3

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import copy
import os

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
FROZEN_EPOCHS = 30
FINE_TUNE_EPOCHS = 70
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
FINE_TUNE_LR = 1e-5
BEST_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/InceptionV3_ft_best_model.pth'
FINAL_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/InceptionV3_ft_final_model.pth'

# Transforms
transform = Compose([
    Resize((224, 224)),  # MobileNetV1 typically uses 224x224 images
    ToTensor(),
    Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset & Dataloaders (Updated paths)
train_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/train', transform=transform)
val_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/valid', transform=transform)
test_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Load pretrained MobileNetV1 and freeze feature layers
base_model = models.inception_v3(pretrained=True)  # MobileNetV1 equivalent is available as MobileNetV2 in torchvision
for param in base_model.parameters():
    param.requires_grad = False  # Freeze all layers initially

# Replace the classifier part of the model (keep global average pooling)
base_model.classifier = nn.Sequential(
    nn.Linear(base_model.classifier[1].in_features, 512),  # base_model.classifier[1].in_features gives the correct input size
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 1),
    nn.Sigmoid()
)

model = base_model.to(device)

# Loss & optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=5, path=BEST_MODEL_PATH):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.best_model = None
        self.path = path

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
            torch.save(self.best_model, self.path)
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# Evaluation Function
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
    return total_loss / len(dataloader.dataset)

# Accuracy Function
def compute_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            preds = (outputs > 0.5).int().squeeze()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
    early_stopping = EarlyStopping(patience=5)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        val_loss = evaluate(model, val_loader, criterion)
        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}")

        if early_stopping(val_loss, model):
            print("Early stopping triggered.")
            break

# Train model with frozen layers
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FROZEN_EPOCHS)

# Load best model before fine-tuning
model.load_state_dict(torch.load(BEST_MODEL_PATH))

# Unfreeze all layers in the classifier and some feature layers
for param in list(model.features.parameters())[-50:]:
    param.requires_grad = True

# Recompile optimizer with updated params
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=FINE_TUNE_LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Fine-tune the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FINE_TUNE_EPOCHS)

# Save final model
torch.save(model.state_dict(), FINAL_MODEL_PATH)
print(f"✅ Final fine-tuned model saved at: {FINAL_MODEL_PATH}")

# Evaluate on test set
test_acc = compute_accuracy(model, test_loader)
print(f"✅ Test Accuracy after Fine-Tuning: {test_acc:.4f}")


c:\Users\vb11574\AppData\Local\miniconda3\envs\salmo\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to C:\Users\vb11574/.cache\torch\hub\checkpoints\inception_v3_google-0cc3c7bd.pth
100.0%


AttributeError: 'Inception3' object has no attribute 'classifier'

# 5 ResNet

## 5.1 ResNet 50

In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import copy
import os

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
FROZEN_EPOCHS = 30
FINE_TUNE_EPOCHS = 70
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
FINE_TUNE_LR = 1e-5
BEST_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/ResNet50_ft_best_model.pth'
FINAL_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/ResNet50_ft_final_model.pth'

# Transforms
transform = Compose([
    Resize((224, 224)),  # ResNet50 typically uses 224x224 images
    ToTensor(),
    Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset & Dataloaders (Updated paths)
train_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/train', transform=transform)
val_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/valid', transform=transform)
test_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Load pretrained ResNet50 and freeze feature layers
base_model = models.resnet50(pretrained=True)  # ResNet50
base_model = base_model.to(device)  # Ensure the model is on the correct device

for param in base_model.parameters():
    param.requires_grad = False  # Freeze all layers initially

# Get the correct number of features from the fc layer (Fully connected layer)
num_ftrs = base_model.fc.in_features  # Get input features to the first Linear layer (fc layer)

# Replace the classifier part of the model (fully connected layer)
base_model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),  # Use num_ftrs for the input size of the linear layer
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 1),
    nn.Sigmoid()
)

model = base_model.to(device)  # Ensure the model is on the correct device

# Loss & optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.fc.parameters(), lr=LEARNING_RATE)  # Update optimizer to work with the new classifier
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=5, path=BEST_MODEL_PATH):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.best_model = None
        self.path = path

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
            torch.save(self.best_model, self.path)
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# Evaluation Function
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
    return total_loss / len(dataloader.dataset)

# Accuracy Function
def compute_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            preds = (outputs > 0.5).int().squeeze()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
    early_stopping = EarlyStopping(patience=5)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        val_loss = evaluate(model, val_loader, criterion)
        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}")

        if early_stopping(val_loss, model):
            print("Early stopping triggered.")
            break

# Train model with frozen layers
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FROZEN_EPOCHS)

# Load best model before fine-tuning
model.load_state_dict(torch.load(BEST_MODEL_PATH))

# Unfreeze all layers in the classifier and some feature layers
for param in list(model.parameters())[-50:]:
    param.requires_grad = True

# Recompile optimizer with updated params
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=FINE_TUNE_LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Fine-tune the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FINE_TUNE_EPOCHS)

# Save final model
torch.save(model.state_dict(), FINAL_MODEL_PATH)
print(f"✅ Final fine-tuned model saved at: {FINAL_MODEL_PATH}")

# Evaluate on test set
test_acc = compute_accuracy(model, test_loader)
print(f"✅ Test Accuracy after Fine-Tuning: {test_acc:.4f}")


Epoch 1/30, Validation Loss: 0.6597
Epoch 2/30, Validation Loss: 0.6553
Epoch 3/30, Validation Loss: 0.6472
Epoch 4/30, Validation Loss: 0.6429
Epoch 5/30, Validation Loss: 0.6403
Epoch 6/30, Validation Loss: 0.6420
Epoch 7/30, Validation Loss: 0.6388
Epoch 8/30, Validation Loss: 0.6366
Epoch 9/30, Validation Loss: 0.6369
Epoch 10/30, Validation Loss: 0.6358
Epoch 11/30, Validation Loss: 0.6391
Epoch 12/30, Validation Loss: 0.6390
Epoch 13/30, Validation Loss: 0.6408
Epoch 14/30, Validation Loss: 0.6449
Epoch 15/30, Validation Loss: 0.6350
Epoch 16/30, Validation Loss: 0.6367
Epoch 17/30, Validation Loss: 0.6345
Epoch 18/30, Validation Loss: 0.6366
Epoch 19/30, Validation Loss: 0.6349
Epoch 20/30, Validation Loss: 0.6362
Epoch 21/30, Validation Loss: 0.6379
Epoch 22/30, Validation Loss: 0.6351
Early stopping triggered.
Epoch 1/70, Validation Loss: 0.6164
Epoch 2/70, Validation Loss: 0.6097
Epoch 3/70, Validation Loss: 0.6108
Epoch 4/70, Validation Loss: 0.6243
Epoch 5/70, Validation Lo

## 5.2 ResNet 101

In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import copy
import os

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
FROZEN_EPOCHS = 30
FINE_TUNE_EPOCHS = 70
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
FINE_TUNE_LR = 1e-5
BEST_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/ResNet50_ft_best_model.pth'
FINAL_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/ResNet50_ft_final_model.pth'

# Transforms
transform = Compose([
    Resize((224, 224)),  # ResNet50 typically uses 224x224 images
    ToTensor(),
    Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset & Dataloaders (Updated paths)
train_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/train', transform=transform)
val_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/valid', transform=transform)
test_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Load pretrained ResNet50 and freeze feature layers
base_model = models.resnet101(pretrained=True)  # ResNet50
base_model = base_model.to(device)  # Ensure the model is on the correct device

for param in base_model.parameters():
    param.requires_grad = False  # Freeze all layers initially

# Get the correct number of features from the fc layer (Fully connected layer)
num_ftrs = base_model.fc.in_features  # Get input features to the first Linear layer (fc layer)

# Replace the classifier part of the model (fully connected layer)
base_model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),  # Use num_ftrs for the input size of the linear layer
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 1),
    nn.Sigmoid()
)

model = base_model.to(device)  # Ensure the model is on the correct device

# Loss & optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.fc.parameters(), lr=LEARNING_RATE)  # Update optimizer to work with the new classifier
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=5, path=BEST_MODEL_PATH):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.best_model = None
        self.path = path

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
            torch.save(self.best_model, self.path)
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# Evaluation Function
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
    return total_loss / len(dataloader.dataset)

# Accuracy Function
def compute_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            preds = (outputs > 0.5).int().squeeze()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
    early_stopping = EarlyStopping(patience=5)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        val_loss = evaluate(model, val_loader, criterion)
        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}")

        if early_stopping(val_loss, model):
            print("Early stopping triggered.")
            break

# Train model with frozen layers
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FROZEN_EPOCHS)

# Load best model before fine-tuning
model.load_state_dict(torch.load(BEST_MODEL_PATH))

# Unfreeze all layers in the classifier and some feature layers
for param in list(model.parameters())[-50:]:
    param.requires_grad = True

# Recompile optimizer with updated params
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=FINE_TUNE_LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Fine-tune the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FINE_TUNE_EPOCHS)

# Save final model
torch.save(model.state_dict(), FINAL_MODEL_PATH)
print(f"✅ Final fine-tuned model saved at: {FINAL_MODEL_PATH}")

# Evaluate on test set
test_acc = compute_accuracy(model, test_loader)
print(f"✅ Test Accuracy after Fine-Tuning: {test_acc:.4f}")


c:\Users\vb11574\AppData\Local\miniconda3\envs\salmo\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\vb11574\AppData\Local\miniconda3\envs\salmo\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to C:\Users\vb11574/.cache\torch\hub\checkpoints\resnet101-63fe2227.pth
100.0%


Epoch 1/30, Validation Loss: 0.6752
Epoch 2/30, Validation Loss: 0.6499
Epoch 3/30, Validation Loss: 0.6462
Epoch 4/30, Validation Loss: 0.6477
Epoch 5/30, Validation Loss: 0.6412
Epoch 6/30, Validation Loss: 0.6438
Epoch 7/30, Validation Loss: 0.6376
Epoch 8/30, Validation Loss: 0.6396
Epoch 9/30, Validation Loss: 0.6311
Epoch 10/30, Validation Loss: 0.6293
Epoch 11/30, Validation Loss: 0.6393
Epoch 12/30, Validation Loss: 0.6339
Epoch 13/30, Validation Loss: 0.6261
Epoch 14/30, Validation Loss: 0.6545
Epoch 15/30, Validation Loss: 0.6269
Epoch 16/30, Validation Loss: 0.6218
Epoch 17/30, Validation Loss: 0.6409
Epoch 18/30, Validation Loss: 0.6232
Epoch 19/30, Validation Loss: 0.6247
Epoch 20/30, Validation Loss: 0.6274
Epoch 21/30, Validation Loss: 0.6236
Early stopping triggered.
Epoch 1/70, Validation Loss: 0.5996
Epoch 2/70, Validation Loss: 0.5816
Epoch 3/70, Validation Loss: 0.5779
Epoch 4/70, Validation Loss: 0.5771
Epoch 5/70, Validation Loss: 0.5892
Epoch 6/70, Validation Los

## 5.3 ResNet 152

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import copy
import os

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
FROZEN_EPOCHS = 30
FINE_TUNE_EPOCHS = 70
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
FINE_TUNE_LR = 1e-5
BEST_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/ResNet50_ft_best_model.pth'
FINAL_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/Models/American_to_African/ResNet50_ft_final_model.pth'

# Transforms
transform = Compose([
    Resize((224, 224)),  # ResNet50 typically uses 224x224 images
    ToTensor(),
    Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset & Dataloaders (Updated paths)
train_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/train', transform=transform)
val_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/valid', transform=transform)
test_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/TransferLearning/DatasetAmericantoAfrican/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Load pretrained ResNet50 and freeze feature layers
base_model = models.resnet152(pretrained=True)  # ResNet50
base_model = base_model.to(device)  # Ensure the model is on the correct device

for param in base_model.parameters():
    param.requires_grad = False  # Freeze all layers initially

# Get the correct number of features from the fc layer (Fully connected layer)
num_ftrs = base_model.fc.in_features  # Get input features to the first Linear layer (fc layer)

# Replace the classifier part of the model (fully connected layer)
base_model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),  # Use num_ftrs for the input size of the linear layer
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 1),
    nn.Sigmoid()
)

model = base_model.to(device)  # Ensure the model is on the correct device

# Loss & optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.fc.parameters(), lr=LEARNING_RATE)  # Update optimizer to work with the new classifier
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=5, path=BEST_MODEL_PATH):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.best_model = None
        self.path = path

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
            torch.save(self.best_model, self.path)
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# Evaluation Function
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
    return total_loss / len(dataloader.dataset)

# Accuracy Function
def compute_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            preds = (outputs > 0.5).int().squeeze()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
    early_stopping = EarlyStopping(patience=5)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        val_loss = evaluate(model, val_loader, criterion)
        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}")

        if early_stopping(val_loss, model):
            print("Early stopping triggered.")
            break

# Train model with frozen layers
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FROZEN_EPOCHS)

# Load best model before fine-tuning
model.load_state_dict(torch.load(BEST_MODEL_PATH))

# Unfreeze all layers in the classifier and some feature layers
for param in list(model.parameters())[-50:]:
    param.requires_grad = True

# Recompile optimizer with updated params
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=FINE_TUNE_LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Fine-tune the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FINE_TUNE_EPOCHS)

# Save final model
torch.save(model.state_dict(), FINAL_MODEL_PATH)
print(f"✅ Final fine-tuned model saved at: {FINAL_MODEL_PATH}")

# Evaluate on test set
test_acc = compute_accuracy(model, test_loader)
print(f"✅ Test Accuracy after Fine-Tuning: {test_acc:.4f}")


c:\Users\vb11574\AppData\Local\miniconda3\envs\salmo\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to C:\Users\vb11574/.cache\torch\hub\checkpoints\resnet152-394f9c45.pth
100.0%


Epoch 1/30, Validation Loss: 0.6597
Epoch 2/30, Validation Loss: 0.6501
Epoch 3/30, Validation Loss: 0.6461
Epoch 4/30, Validation Loss: 0.6429
Epoch 5/30, Validation Loss: 0.6411
Epoch 6/30, Validation Loss: 0.6444
Epoch 7/30, Validation Loss: 0.6617
Epoch 8/30, Validation Loss: 0.6445
Epoch 9/30, Validation Loss: 0.6466
Epoch 10/30, Validation Loss: 0.6320
Epoch 11/30, Validation Loss: 0.6331
Epoch 12/30, Validation Loss: 0.6344
Epoch 13/30, Validation Loss: 0.6315
Epoch 14/30, Validation Loss: 0.6308
Epoch 15/30, Validation Loss: 0.6306
Epoch 16/30, Validation Loss: 0.6313
Epoch 17/30, Validation Loss: 0.6300
Epoch 18/30, Validation Loss: 0.6304
Epoch 19/30, Validation Loss: 0.6291
Epoch 20/30, Validation Loss: 0.6298
Epoch 21/30, Validation Loss: 0.6305
Epoch 22/30, Validation Loss: 0.6283
Epoch 23/30, Validation Loss: 0.6294
Epoch 24/30, Validation Loss: 0.6307
Epoch 25/30, Validation Loss: 0.6316
Epoch 26/30, Validation Loss: 0.6275
Epoch 27/30, Validation Loss: 0.6296
Epoch 28/3

# 6 EfficientNet

## 6.1 EfficientNet B0

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import copy
import os

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
FROZEN_EPOCHS = 30
FINE_TUNE_EPOCHS = 70
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
FINE_TUNE_LR = 1e-5
BEST_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/Models/TransferLearning/Models/American_to_African/EfficientNetB0_ft_best_model.pth'
FINAL_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/Models/TransferLearning/Models/American_to_African/EfficientNetB0_ft_final_model.pth'

# Transforms
transform = Compose([
    Resize((224, 224)),  # MobileNetV1 typically uses 224x224 images
    ToTensor(),
    Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset & Dataloaders (Updated paths)
train_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/Models/TransferLearning/DatasetAmericantoAfrican/train', transform=transform)
val_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/Models/TransferLearning/DatasetAmericantoAfrican/valid', transform=transform)
test_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/Models/TransferLearning/DatasetAmericantoAfrican/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Load pretrained MobileNetV1 and freeze feature layers
base_model = models.efficientnet_b0(pretrained=True)  # MobileNetV1 equivalent is available as MobileNetV2 in torchvision
for param in base_model.parameters():
    param.requires_grad = False  # Freeze all layers initially

# Replace the classifier part of the model (keep global average pooling)
base_model.classifier = nn.Sequential(
    nn.Linear(base_model.classifier[1].in_features, 512),  # base_model.classifier[1].in_features gives the correct input size
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 1),
    nn.Sigmoid()
)

model = base_model.to(device)

# Loss & optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=5, path=BEST_MODEL_PATH):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.best_model = None
        self.path = path

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
            torch.save(self.best_model, self.path)
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# Evaluation Function
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
    return total_loss / len(dataloader.dataset)

# Accuracy Function
def compute_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            preds = (outputs > 0.5).int().squeeze()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
    early_stopping = EarlyStopping(patience=5)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        val_loss = evaluate(model, val_loader, criterion)
        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}")

        if early_stopping(val_loss, model):
            print("Early stopping triggered.")
            break

# Train model with frozen layers
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FROZEN_EPOCHS)

# Load best model before fine-tuning
model.load_state_dict(torch.load(BEST_MODEL_PATH))

# Unfreeze all layers in the classifier and some feature layers
for param in list(model.features.parameters())[-50:]:
    param.requires_grad = True

# Recompile optimizer with updated params
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=FINE_TUNE_LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Fine-tune the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FINE_TUNE_EPOCHS)

# Save final model
torch.save(model.state_dict(), FINAL_MODEL_PATH)
print(f"✅ Final fine-tuned model saved at: {FINAL_MODEL_PATH}")

# Evaluate on test set
test_acc = compute_accuracy(model, test_loader)
print(f"✅ Test Accuracy after Fine-Tuning: {test_acc:.4f}")


Epoch 1/30, Validation Loss: 0.6591
Epoch 2/30, Validation Loss: 0.6533
Epoch 3/30, Validation Loss: 0.6459
Epoch 4/30, Validation Loss: 0.6415
Epoch 5/30, Validation Loss: 0.6394
Epoch 6/30, Validation Loss: 0.6388
Epoch 7/30, Validation Loss: 0.6338
Epoch 8/30, Validation Loss: 0.6343
Epoch 9/30, Validation Loss: 0.6273
Epoch 10/30, Validation Loss: 0.6391
Epoch 11/30, Validation Loss: 0.6298
Epoch 12/30, Validation Loss: 0.6296
Epoch 13/30, Validation Loss: 0.6198
Epoch 14/30, Validation Loss: 0.6166
Epoch 15/30, Validation Loss: 0.6189
Epoch 16/30, Validation Loss: 0.6213
Epoch 17/30, Validation Loss: 0.6166
Epoch 18/30, Validation Loss: 0.6146
Epoch 19/30, Validation Loss: 0.6246
Epoch 20/30, Validation Loss: 0.6164
Epoch 21/30, Validation Loss: 0.6249
Epoch 22/30, Validation Loss: 0.6210
Epoch 23/30, Validation Loss: 0.6190
Early stopping triggered.
Epoch 1/70, Validation Loss: 0.6102
Epoch 2/70, Validation Loss: 0.6134
Epoch 3/70, Validation Loss: 0.6106
Epoch 4/70, Validation L

## 6.2 EfficientNet B3

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import copy
import os

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
FROZEN_EPOCHS = 30
FINE_TUNE_EPOCHS = 70
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
FINE_TUNE_LR = 1e-5
BEST_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/Models/TransferLearning/Models/American_to_African/EfficientNetB3_ft_best_model.pth'
FINAL_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/Models/TransferLearning/Models/American_to_African/EfficientNetB3_ft_final_model.pth'

# Transforms
transform = Compose([
    Resize((224, 224)),  # MobileNetV1 typically uses 224x224 images
    ToTensor(),
    Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset & Dataloaders (Updated paths)
train_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/Models/TransferLearning/DatasetAmericantoAfrican/train', transform=transform)
val_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/Models/TransferLearning/DatasetAmericantoAfrican/valid', transform=transform)
test_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/Models/TransferLearning/DatasetAmericantoAfrican/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Load pretrained MobileNetV1 and freeze feature layers
base_model = models.efficientnet_b3(pretrained=True)  # MobileNetV1 equivalent is available as MobileNetV2 in torchvision
for param in base_model.parameters():
    param.requires_grad = False  # Freeze all layers initially

# Replace the classifier part of the model (keep global average pooling)
base_model.classifier = nn.Sequential(
    nn.Linear(base_model.classifier[1].in_features, 512),  # base_model.classifier[1].in_features gives the correct input size
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 1),
    nn.Sigmoid()
)

model = base_model.to(device)

# Loss & optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=5, path=BEST_MODEL_PATH):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.best_model = None
        self.path = path

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
            torch.save(self.best_model, self.path)
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# Evaluation Function
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
    return total_loss / len(dataloader.dataset)

# Accuracy Function
def compute_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            preds = (outputs > 0.5).int().squeeze()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
    early_stopping = EarlyStopping(patience=5)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        val_loss = evaluate(model, val_loader, criterion)
        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}")

        if early_stopping(val_loss, model):
            print("Early stopping triggered.")
            break

# Train model with frozen layers
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FROZEN_EPOCHS)

# Load best model before fine-tuning
model.load_state_dict(torch.load(BEST_MODEL_PATH))

# Unfreeze all layers in the classifier and some feature layers
for param in list(model.features.parameters())[-50:]:
    param.requires_grad = True

# Recompile optimizer with updated params
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=FINE_TUNE_LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Fine-tune the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FINE_TUNE_EPOCHS)

# Save final model
torch.save(model.state_dict(), FINAL_MODEL_PATH)
print(f"✅ Final fine-tuned model saved at: {FINAL_MODEL_PATH}")

# Evaluate on test set
test_acc = compute_accuracy(model, test_loader)
print(f"✅ Test Accuracy after Fine-Tuning: {test_acc:.4f}")


c:\Users\vb11574\AppData\Local\miniconda3\envs\salmo\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B3_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b3_rwightman-b3899882.pth" to C:\Users\vb11574/.cache\torch\hub\checkpoints\efficientnet_b3_rwightman-b3899882.pth
100%|██████████| 47.2M/47.2M [00:00<00:00, 112MB/s] 


Epoch 1/30, Validation Loss: 0.6672
Epoch 2/30, Validation Loss: 0.6621
Epoch 3/30, Validation Loss: 0.6553
Epoch 4/30, Validation Loss: 0.6517
Epoch 5/30, Validation Loss: 0.6481
Epoch 6/30, Validation Loss: 0.6453
Epoch 7/30, Validation Loss: 0.6429
Epoch 8/30, Validation Loss: 0.6432
Epoch 9/30, Validation Loss: 0.6373
Epoch 10/30, Validation Loss: 0.6362
Epoch 11/30, Validation Loss: 0.6335
Epoch 12/30, Validation Loss: 0.6326
Epoch 13/30, Validation Loss: 0.6322
Epoch 14/30, Validation Loss: 0.6340
Epoch 15/30, Validation Loss: 0.6265
Epoch 16/30, Validation Loss: 0.6251
Epoch 17/30, Validation Loss: 0.6350
Epoch 18/30, Validation Loss: 0.6384
Epoch 19/30, Validation Loss: 0.6314
Epoch 20/30, Validation Loss: 0.6325
Epoch 21/30, Validation Loss: 0.6342
Early stopping triggered.
Epoch 1/70, Validation Loss: 0.6273
Epoch 2/70, Validation Loss: 0.6264
Epoch 3/70, Validation Loss: 0.6266
Epoch 4/70, Validation Loss: 0.6230
Epoch 5/70, Validation Loss: 0.6219
Epoch 6/70, Validation Los

## 6.3 EfficientNet B7

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import copy
import os

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
FROZEN_EPOCHS = 30
FINE_TUNE_EPOCHS = 70
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
FINE_TUNE_LR = 1e-5
BEST_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/Models/TransferLearning/Models/American_to_African/EfficientNetB7_ft_best_model.pth'
FINAL_MODEL_PATH = 'C:/Users/vb11574/Desktop/Salmonella_Project/Models/TransferLearning/Models/American_to_African/EfficientNetB7_ft_final_model.pth'

# Transforms
transform = Compose([
    Resize((224, 224)),  # MobileNetV1 typically uses 224x224 images
    ToTensor(),
    Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset & Dataloaders (Updated paths)
train_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/Models/TransferLearning/DatasetAmericantoAfrican/train', transform=transform)
val_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/Models/TransferLearning/DatasetAmericantoAfrican/valid', transform=transform)
test_dataset = ImageFolder('C:/Users/vb11574/Desktop/Salmonella_Project/Models/TransferLearning/DatasetAmericantoAfrican/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Load pretrained MobileNetV1 and freeze feature layers
base_model = models.efficientnet_b7(pretrained=True)  # MobileNetV1 equivalent is available as MobileNetV2 in torchvision
for param in base_model.parameters():
    param.requires_grad = False  # Freeze all layers initially

# Replace the classifier part of the model (keep global average pooling)
base_model.classifier = nn.Sequential(
    nn.Linear(base_model.classifier[1].in_features, 512),  # base_model.classifier[1].in_features gives the correct input size
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 1),
    nn.Sigmoid()
)

model = base_model.to(device)

# Loss & optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=5, path=BEST_MODEL_PATH):
        self.patience = patience
        self.counter = 0
        self.best_loss = float('inf')
        self.best_model = None
        self.path = path

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
            torch.save(self.best_model, self.path)
            self.counter = 0
        else:
            self.counter += 1
        return self.counter >= self.patience

# Evaluation Function
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
    return total_loss / len(dataloader.dataset)

# Accuracy Function
def compute_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            preds = (outputs > 0.5).int().squeeze()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
    early_stopping = EarlyStopping(patience=5)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        val_loss = evaluate(model, val_loader, criterion)
        scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}")

        if early_stopping(val_loss, model):
            print("Early stopping triggered.")
            break

# Train model with frozen layers
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FROZEN_EPOCHS)

# Load best model before fine-tuning
model.load_state_dict(torch.load(BEST_MODEL_PATH))

# Unfreeze all layers in the classifier and some feature layers
for param in list(model.features.parameters())[-50:]:
    param.requires_grad = True

# Recompile optimizer with updated params
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=FINE_TUNE_LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Fine-tune the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, FINE_TUNE_EPOCHS)

# Save final model
torch.save(model.state_dict(), FINAL_MODEL_PATH)
print(f"✅ Final fine-tuned model saved at: {FINAL_MODEL_PATH}")

# Evaluate on test set
test_acc = compute_accuracy(model, test_loader)
print(f"✅ Test Accuracy after Fine-Tuning: {test_acc:.4f}")


c:\Users\vb11574\AppData\Local\miniconda3\envs\salmo\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B7_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B7_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b7_lukemelas-c5b4e57e.pth" to C:\Users\vb11574/.cache\torch\hub\checkpoints\efficientnet_b7_lukemelas-c5b4e57e.pth
100%|██████████| 255M/255M [00:02<00:00, 117MB/s]  


Epoch 1/30, Validation Loss: 0.6837
Epoch 2/30, Validation Loss: 0.6662
Epoch 3/30, Validation Loss: 0.6580
Epoch 4/30, Validation Loss: 0.6538
Epoch 5/30, Validation Loss: 0.6498
Epoch 6/30, Validation Loss: 0.6474
Epoch 7/30, Validation Loss: 0.6470
Epoch 8/30, Validation Loss: 0.6461
Epoch 9/30, Validation Loss: 0.6475
Epoch 10/30, Validation Loss: 0.6472
Epoch 11/30, Validation Loss: 0.6463
Epoch 12/30, Validation Loss: 0.6432
Epoch 13/30, Validation Loss: 0.6442
Epoch 14/30, Validation Loss: 0.6473
Epoch 15/30, Validation Loss: 0.6448
Epoch 16/30, Validation Loss: 0.6428
Epoch 17/30, Validation Loss: 0.6405
Epoch 18/30, Validation Loss: 0.6419
Epoch 19/30, Validation Loss: 0.6416
Epoch 20/30, Validation Loss: 0.6435
Epoch 21/30, Validation Loss: 0.6422
Epoch 22/30, Validation Loss: 0.6418
Early stopping triggered.
Epoch 1/70, Validation Loss: 0.6349
Epoch 2/70, Validation Loss: 0.6291
Epoch 3/70, Validation Loss: 0.6267
Epoch 4/70, Validation Loss: 0.6265
Epoch 5/70, Validation Lo